In [1]:
#suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pandas as pd
import glob
import os

Using TensorFlow backend.


In [3]:
def write_weights(raw_text,seq_length,epoch_num,num,verbose=0):
    #get chars in raw_text
    chars = sorted(list(set(raw_text)))
    
    #get mapping of char to int value and vice versa
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    
    #print out summary of data
    n_chars = len(raw_text)
    n_vocab = len(chars)
    print("Total Characters: ", n_chars)
    print("Total Vocab (Unique Characters): ", n_vocab)
    
    #get X and Y of data (sequence in and next character)
    #X will be a sequence of characters with length seq_length (given)
    #Y will be the corresponding next character
    dataX = []
    dataY = []
    for i in range(0, n_chars - seq_length, 1):
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)
    #num of patters is the number of text sequences of length seq_length we have
    print("Total Patterns: ", n_patterns,'\n')
    
    # reshape X to be [samples, time steps, features]
    X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
    
    # normalize X (really simple scaler)
    X = X / float(n_vocab)
    
    # do one hot encoding on the output variable
    y = np_utils.to_categorical(dataY)
    
    # define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # define the checkpoint, make file names standard
    filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    
    # fit the model
    model.fit(X, y, epochs=epoch_num, batch_size=128, callbacks=callbacks_list)
    
    # define the LSTM model - part 2
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    
    #get latest file (newest weight)
    list_of_files = glob.glob('../twilightvalefalls/*.hdf5')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # load the network weights
    filename = latest_file
    model.load_weights(filename)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # pick a random seed num times (new one for each text generated)
    seeds = []
    for n in range(num):
        start = numpy.random.randint(0, len(dataX)-1)
        pattern = dataX[start]
        s = ''.join([int_to_char[value] for value in pattern])
        if verbose > 0:
            print("Seed #%d:" %(n+1))
            print("\"", s, "\"")
        seeds.append(pattern)
        
    gens = []
    for seed in seeds:
        pattern = seed
        # generate characters
        for i in range(seq_length):
            x = numpy.reshape(pattern, (1, len(pattern), 1))
            x = x / float(n_vocab)
            prediction = model.predict(x, verbose=0)
            index = numpy.argmax(prediction)
            result = int_to_char[index]
            seq_in = [int_to_char[value] for value in pattern]
            pattern.append(index)
            pattern = pattern[1:len(pattern)]
            
        gens.append(pattern)
        
    for text in gens:
        if verbose > 0:
            print(''.join([int_to_char[value] for value in text]))
        
    return [''.join([int_to_char[value] for value in text]) for text in gens]

In [4]:
#RStories RNN
txt_files = glob.glob("rStories/*.txt")
rstories_fulltext = ''
for f in txt_files:
    file = open(f,encoding='utf-8') 
    st = file.read() 
    rstories_fulltext = rstories_fulltext + ' ' + st
    file.close()

In [5]:
rstories_fulltext = rstories_fulltext.replace('\n',' ').replace('        ','').strip()

In [6]:
rstories_texts = write_weights(rstories_fulltext,100,200,10,verbose=1)

Total Characters:  26885
Total Vocab (Unique Characters):  66
Total Patterns:  26785 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
26785/26785 [==============================] - 74s 3ms/step - loss: 3.1331

Epoch 00001: loss improved from inf to 3.13311, saving model to weights-improvement-01-3.1331.hdf5
Epoch 2/200
26785/26785 [==============================] - 72s 3ms/step - loss: 3.0736

Epoch 00002: loss improved from 3.13311 to 3.07360, saving model to weights-improvement-02-3.0736.hdf5
Epoch 3/200
26785/26785 [==============================] - 74s 3ms/step - loss: 3.0390

Epoch 00003: loss improved from 3.07360 to 3.03900, saving model to weights-improvement-03-3.0390.hdf5
Epoch 4/200
26785/26785 [==============================] - 72s 3ms/step - loss: 2.9461

Epoch 00004: loss imp

26785/26785 [==============================] - 72s 3ms/step - loss: 2.1973

Epoch 00038: loss improved from 2.23584 to 2.19726, saving model to weights-improvement-38-2.1973.hdf5
Epoch 39/200
26785/26785 [==============================] - 73s 3ms/step - loss: 2.1653

Epoch 00039: loss improved from 2.19726 to 2.16530, saving model to weights-improvement-39-2.1653.hdf5
Epoch 40/200
26785/26785 [==============================] - 72s 3ms/step - loss: 2.1259

Epoch 00040: loss improved from 2.16530 to 2.12591, saving model to weights-improvement-40-2.1259.hdf5
Epoch 41/200
26785/26785 [==============================] - 72s 3ms/step - loss: 2.0932

Epoch 00041: loss improved from 2.12591 to 2.09317, saving model to weights-improvement-41-2.0932.hdf5
Epoch 42/200
26785/26785 [==============================] - 72s 3ms/step - loss: 2.0677

Epoch 00042: loss improved from 2.09317 to 2.06771, saving model to weights-improvement-42-2.0677.hdf5
Epoch 43/200
26785/26785 [===========================

26785/26785 [==============================] - 113s 4ms/step - loss: 1.2404

Epoch 00082: loss improved from 1.25437 to 1.24042, saving model to weights-improvement-82-1.2404.hdf5
Epoch 83/200
26785/26785 [==============================] - 104s 4ms/step - loss: 1.2528

Epoch 00083: loss did not improve from 1.24042
Epoch 84/200
26785/26785 [==============================] - 76s 3ms/step - loss: 1.2505

Epoch 00084: loss did not improve from 1.24042
Epoch 85/200
26785/26785 [==============================] - 71s 3ms/step - loss: 1.2164

Epoch 00085: loss improved from 1.24042 to 1.21638, saving model to weights-improvement-85-1.2164.hdf5
Epoch 86/200
26785/26785 [==============================] - 72s 3ms/step - loss: 1.2379

Epoch 00086: loss did not improve from 1.21638
Epoch 87/200
26785/26785 [==============================] - 75s 3ms/step - loss: 1.1754

Epoch 00087: loss improved from 1.21638 to 1.17538, saving model to weights-improvement-87-1.1754.hdf5
Epoch 88/200
26785/26785 [=

26785/26785 [==============================] - 75s 3ms/step - loss: 2.7102

Epoch 00191: loss did not improve from 0.76817
Epoch 192/200
26785/26785 [==============================] - 76s 3ms/step - loss: 2.6849

Epoch 00192: loss did not improve from 0.76817
Epoch 193/200
26785/26785 [==============================] - 74s 3ms/step - loss: 2.6658

Epoch 00193: loss did not improve from 0.76817
Epoch 194/200
26785/26785 [==============================] - 73s 3ms/step - loss: 2.6394

Epoch 00194: loss did not improve from 0.76817
Epoch 195/200
26785/26785 [==============================] - 74s 3ms/step - loss: 2.6183

Epoch 00195: loss did not improve from 0.76817
Epoch 196/200
26785/26785 [==============================] - 73s 3ms/step - loss: 2.6005

Epoch 00196: loss did not improve from 0.76817
Epoch 197/200
26785/26785 [==============================] - 73s 3ms/step - loss: 2.5729

Epoch 00197: loss did not improve from 0.76817
Epoch 198/200
26785/26785 [============================

In [7]:
hhg_fulltext = ''
f = 'hhg.txt'
file = open(f)
hhg_fulltext = file.read()
file.close()

In [8]:
hhg_fulltext = hhg_fulltext.replace('THE HITCHHIKER\'S GUIDE TO THE GALAXY\nBY DOUGLAS ADAMS\n','').replace('\n',' ')

In [9]:
hhg_texts = write_weights(hhg_fulltext,10,200,25,verbose=1)

Total Characters:  266523
Total Vocab (Unique Characters):  86
Total Patterns:  266513 

Epoch 1/200
266513/266513 [==============================] - 96s 360us/step - loss: 3.0048

Epoch 00001: loss improved from inf to 3.00484, saving model to weights-improvement-01-3.0048.hdf5
Epoch 2/200
266513/266513 [==============================] - 86s 322us/step - loss: 2.8487

Epoch 00002: loss improved from 3.00484 to 2.84868, saving model to weights-improvement-02-2.8487.hdf5
Epoch 3/200
266513/266513 [==============================] - 81s 303us/step - loss: 2.7818

Epoch 00003: loss improved from 2.84868 to 2.78184, saving model to weights-improvement-03-2.7818.hdf5
Epoch 4/200
266513/266513 [==============================] - 81s 304us/step - loss: 2.7337

Epoch 00004: loss improved from 2.78184 to 2.73365, saving model to weights-improvement-04-2.7337.hdf5
Epoch 5/200
266513/266513 [==============================] - 82s 307us/step - loss: 2.6887

Epoch 00005: loss improved from 2.73365 to 

266513/266513 [==============================] - 82s 308us/step - loss: 1.7848

Epoch 00084: loss improved from 1.79119 to 1.78478, saving model to weights-improvement-84-1.7848.hdf5
Epoch 85/200
266513/266513 [==============================] - 82s 308us/step - loss: 1.7851

Epoch 00085: loss did not improve from 1.78478
Epoch 86/200
266513/266513 [==============================] - 82s 307us/step - loss: 1.7826

Epoch 00086: loss improved from 1.78478 to 1.78257, saving model to weights-improvement-86-1.7826.hdf5
Epoch 87/200
266513/266513 [==============================] - 82s 307us/step - loss: 1.7775

Epoch 00087: loss improved from 1.78257 to 1.77752, saving model to weights-improvement-87-1.7775.hdf5
Epoch 88/200
266513/266513 [==============================] - 82s 306us/step - loss: 1.7749

Epoch 00088: loss improved from 1.77752 to 1.77486, saving model to weights-improvement-88-1.7749.hdf5
Epoch 89/200
266513/266513 [==============================] - 83s 310us/step - loss: 1.77

266513/266513 [==============================] - 89s 335us/step - loss: 1.6067

Epoch 00171: loss did not improve from 1.60538
Epoch 172/200
266513/266513 [==============================] - 89s 336us/step - loss: 1.6032

Epoch 00172: loss improved from 1.60538 to 1.60316, saving model to weights-improvement-172-1.6032.hdf5
Epoch 173/200
266513/266513 [==============================] - 90s 338us/step - loss: 1.6046

Epoch 00173: loss did not improve from 1.60316
Epoch 174/200
266513/266513 [==============================] - 89s 335us/step - loss: 1.6005

Epoch 00174: loss improved from 1.60316 to 1.60053, saving model to weights-improvement-174-1.6005.hdf5
Epoch 175/200
266513/266513 [==============================] - 89s 335us/step - loss: 1.5997

Epoch 00175: loss improved from 1.60053 to 1.59974, saving model to weights-improvement-175-1.5997.hdf5
Epoch 176/200
266513/266513 [==============================] - 90s 339us/step - loss: 1.59911s 

Epoch 00176: loss improved from 1.59974 t